In [3]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
import pandas as pd
df = pd.read_csv("raw/sspgdp.csv")
df.head()

,Model,Scenario,Region,Variable,Unit,2010,2015,2020,2025,2030,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
0,OECD ENV-Growth 2023,SSP1,Aruba,GDP|PPP,billion USD_2017/yr,NaN,NaN,3.11606,4.60237,4.88357,...,7.47681,8.17492,8.92051,9.6737,10.4262,11.1708,11.892,12.5779,13.2108,13.7626
1,OECD ENV-Growth 2023,SSP1,Angola,GDP|PPP,billion USD_2017/yr,NaN,NaN,201.39500,227.03700,272.62800,...,869.50000,1101.90000,1383.07000,1712.6100,2090.4100,2511.5800,2971.550,3466.5300,3989.2000,4529.6700
2,OECD ENV-Growth 2023,SSP1,Albania,GDP|PPP,billion USD_2017/yr,NaN,NaN,38.61860,48.82710,58.06450,...,103.25700,108.27400,113.04700,117.7460,121.6650,124.7700,127.062,128.6320,129.4820,129.8160
3,OECD ENV-Growth 2023,SSP1,United Arab Emirates,GDP|PPP,billion USD_2017/yr,NaN,NaN,628.45500,792.17400,983.03500,...,1529.24000,1563.54000,1587.76000,1594.2300,1579.5600,1550.2200,1508.370,1459.3600,1409.4100,1364.0900
4,OECD ENV-Growth 2023,SSP1,Argentina,GDP|PPP,billion USD_2017/yr,NaN,NaN,750.58000,861.74100,976.73200,...,1835.60000,2056.50000,2287.45000,2522.0300,2751.2800,2969.7200,3174.740,3363.2300,3529.7600,3667.2700


In [8]:
df = df.drop(columns=['Model', 'Variable', 'Unit', "2010", "2015"])

In [4]:
df = df.groupby(['Scenario', 'Region'], as_index=False).sum()

In [5]:
scenario_dfs = {
    scenario: df[df['Scenario'] == scenario].reset_index(drop=True)
    for scenario in df['Scenario'].unique()
}

In [7]:
ssp1_df = scenario_dfs['SSP1']
ssp2_df = scenario_dfs['SSP2']
ssp3_df = scenario_dfs['SSP3']
ssp4_df = scenario_dfs['SSP4']
ssp5_df = scenario_dfs['SSP5']

In [9]:
def interpolate_annual(df):
    df_long = df.melt(id_vars=['Scenario', 'Region'], var_name='Year', value_name='GDP')
    df_long['Year'] = df_long['Year'].astype(int)

    full_years = pd.DataFrame({'Year': range(df_long['Year'].min(), df_long['Year'].max() + 1)})

    df_filled = (
        df_long.groupby(['Scenario', 'Region'], group_keys=False)
        .apply(lambda group: (
            pd.merge(full_years, group, on='Year', how='left')
            .assign(Scenario=group['Scenario'].iloc[0], Region=group['Region'].iloc[0])
            .interpolate(method='linear')
        ))
        .reset_index(drop=True)
    )

    df_interp = df_filled.pivot(index=['Scenario', 'Region'], columns='Year', values='GDP').reset_index()
    df_interp.columns.name = None
    df_interp.columns = df_interp.columns.astype(str)
    return df_interp

ssp1_interp = interpolate_annual(ssp1_df)
ssp2_interp = interpolate_annual(ssp2_df)
ssp3_interp = interpolate_annual(ssp3_df)
ssp4_interp = interpolate_annual(ssp4_df)
ssp5_interp = interpolate_annual(ssp5_df)

ssp1_interp.to_csv("data/ssp1_gdp.csv", index=False)
ssp2_interp.to_csv("data/ssp2_gdp.csv", index=False)
ssp3_interp.to_csv("data/ssp3_gdp.csv", index=False)
ssp4_interp.to_csv("data/ssp4_gdp.csv", index=False)
ssp5_interp.to_csv("data/ssp5_gdp.csv", index=False)

In [10]:
ssp1_interp.head()

,Scenario,Region,2020,2021,2022,2023,2024,2025,2026,2027,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,SSP1,Albania,38.61860,40.660300,42.702000,44.743700,46.785400,48.8271,50.674580,52.522060,...,128.80200,128.97200,129.14200,129.31200,129.4820,129.54880,129.61560,129.68240,129.74920,129.8160
1,SSP1,Algeria,463.72200,479.635200,495.548400,511.461600,527.374800,543.2880,553.808200,564.328400,...,3474.34000,3555.30000,3636.26000,3717.22000,3798.1800,3883.17200,3968.16400,4053.15600,4138.14800,4223.1400
2,SSP1,Angola,201.39500,206.523400,211.651800,216.780200,221.908600,227.0370,236.155200,245.273400,...,3571.06400,3675.59800,3780.13200,3884.66600,3989.2000,4097.29400,4205.38800,4313.48200,4421.57600,4529.6700
3,SSP1,Antigua and Barbuda,1.74094,1.859352,1.977764,2.096176,2.214588,2.3330,2.404894,2.476788,...,11.08038,11.19996,11.31954,11.43912,11.5587,11.66084,11.76298,11.86512,11.96726,12.0694
4,SSP1,Argentina,750.58000,772.812200,795.044400,817.276600,839.508800,861.7410,884.739200,907.737400,...,3396.53600,3429.84200,3463.14800,3496.45400,3529.7600,3557.26200,3584.76400,3612.26600,3639.76800,3667.2700


In [11]:
dfpop=pd.read_csv("raw/population.csv")

In [12]:
dfpop = dfpop.drop(columns=['Model', 'Variable', 'Unit'])

In [13]:
dfpop= dfpop.groupby(['Scenario', 'Region'], as_index=False).sum()

In [18]:
scenariopop_dfs = {
    scenario: dfpop[dfpop['Scenario'] == scenario].reset_index(drop=True)
    for scenario in df['Scenario'].unique()
}

In [19]:
ssp1_pop = scenariopop_dfs['SSP1']
ssp2_pop = scenariopop_dfs['SSP2']
ssp3_pop = scenariopop_dfs['SSP3']
ssp4_pop= scenariopop_dfs['SSP4']
ssp5_pop= scenariopop_dfs['SSP5']

In [20]:
ssp1p_interp = interpolate_annual(ssp1_pop)
ssp2p_interp = interpolate_annual(ssp2_pop)
ssp3p_interp = interpolate_annual(ssp3_pop)
ssp4p_interp = interpolate_annual(ssp4_pop)
ssp5p_interp = interpolate_annual(ssp5_pop)

ssp1p_interp.to_csv("data/ssp1_pop.csv", index=False)
ssp2p_interp.to_csv("data/ssp2_pop.csv", index=False)
ssp3p_interp.to_csv("data/ssp3_pop.csv", index=False)
ssp4p_interp.to_csv("data/ssp4_pop.csv", index=False)
ssp5p_interp.to_csv("data/ssp5_pop.csv", index=False)

In [21]:
ssp1p_interp.head()

,Scenario,Region,2020,2021,2022,2023,2024,2025,2026,2027,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,SSP1,Afghanistan,38.326031,39.419564,40.513097,41.606629,42.700162,43.793695,44.882668,45.971641,...,79.569255,79.328720,79.088185,78.847650,78.607115,78.282681,77.958248,77.633814,77.309381,76.984947
1,SSP1,Albania,2.872130,2.856190,2.840251,2.824311,2.808372,2.792432,2.776861,2.761290,...,1.491476,1.475693,1.459911,1.444128,1.428346,1.413020,1.397693,1.382367,1.367040,1.351714
2,SSP1,Algeria,43.090992,43.770728,44.450464,45.130199,45.809935,46.489671,47.028835,47.567998,...,54.407648,54.199993,53.992339,53.784684,53.577029,53.350020,53.123010,52.896001,52.668991,52.441982
3,SSP1,Angola,32.899002,33.899600,34.900197,35.900795,36.901392,37.901990,38.858763,39.815535,...,73.691720,73.505867,73.320014,73.134161,72.948308,72.662869,72.377430,72.091990,71.806551,71.521112
4,SSP1,Antigua and Barbuda,0.092375,0.092961,0.093547,0.094133,0.094719,0.095305,0.095772,0.096240,...,0.085351,0.084595,0.083840,0.083084,0.082328,0.081507,0.080687,0.079866,0.079046,0.078225


In [22]:
def compute_growth(df):
    growth_df = df.copy()
    meta_cols = ['Scenario', 'Region']
    year_cols = [col for col in df.columns if col not in meta_cols]
    growth_rates = df[year_cols].pct_change(axis=1)
    
    for col in meta_cols:
        growth_rates[col] = df[col]
    
    cols_order = meta_cols + year_cols
    return growth_rates[cols_order]

In [23]:
ssp1_growth = compute_growth(ssp1_interp)
ssp2_growth = compute_growth(ssp2_interp)
ssp3_growth = compute_growth(ssp3_interp)
ssp4_growth = compute_growth(ssp4_interp)
ssp5_growth = compute_growth(ssp5_interp)

In [25]:
ssp1_growth.to_csv("data/ssp1_growth.csv", index=False)
ssp2_growth.to_csv("data/ssp2_growth.csv", index=False)
ssp3_growth.to_csv("data/ssp3_growth.csv", index=False)
ssp4_growth.to_csv("data/ssp4_growth.csv", index=False)
ssp5_growth.to_csv("data/ssp5_growth.csv", index=False)